In [1]:
from selenium import webdriver
from time import sleep
import threading
import re 
import urllib 
from bs4 import BeautifulSoup

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
items = pd.read_csv('data/item.csv')
items_detail= pd.read_csv('data/item_detail.csv')

In [4]:
len(items), len(items_detail)

(5180, 5180)

In [5]:
merged = pd.merge(items, items_detail, how='inner', on='id_item')

In [6]:
def getname(name, brand): 
    return name.replace(str(brand), '')

In [7]:
merged['item_name'] = merged.apply(lambda row: getname(row['name'], row['brand']), axis=1)

In [175]:
def get_comment(id_item, keyword, driver):
     
    ukey = urllib.parse.quote(keyword.encode('utf8'), '/:') 
    
    url = "https://msearch.shopping.naver.com/search/all.nhn?origQuery={}&pagingIndex=1&viewType=lst&sort=rel&showFilter=true&frm=NVSHSRC&selectedFilterTab=price&query={}".format(ukey,ukey)
       
    driver.get(url) 
    sleep(1)
     
    reviews = driver.find_elements_by_css_selector('div.info_main .info_buy .grade') 
    rows = []
    if len(reviews) > 0: 
        for review in reviews:
            
            parent = review.find_element_by_xpath(".//ancestor::div[@class='info_main']")
            link = parent.find_element_by_css_selector('a.a_link').get_attribute('href')
            num = review.find_element_by_css_selector('.num').text
            cmts = review.find_element_by_css_selector('.help').text
            
            rows.append([id_item, link, num, cmts, keyword])
    else :
        reviews = driver.find_elements_by_css_selector('div.info_main .info_buy .piece') 
        
        revcnt = 0
        
        for review in reviews:
            if review.text.startswith('리뷰'): 
                cmts = review.text.split('리뷰')[1] 
                revcnt += 1 
                parent = review.find_element_by_xpath(".//ancestor::div[@class='info_main']")
                link = parent.find_element_by_css_selector('a.a_link').get_attribute('href')
                #print(link)
                rows.append([id_item, link, np.nan, cmts, keyword])
        
        if revcnt == 0:
            rows.append([id_item, np.nan, np.nan, np.nan, keyword])
    return rows
    
def get_comment_group(items, names):
    driver =  webdriver.PhantomJS()
    for item, name in zip(items, names): 
         
        review_links = get_comment(item, name, driver)
        items_comments.extend(review_links)
    driver.close() 

In [176]:
!killall phantomjs

No matching processes belonging to you were found


In [177]:
items_comments = []
for cnt in range(1, 11): 
    start = 518 * (cnt-1)
    end = 518 * cnt-1
    items = merged.loc[start:end]['id_item'].values
    f_names = merged.loc[start:end]['fullname'].values
  
    th = threading.Thread(target=get_comment_group, args=(items,f_names, ))
    th.start()

In [197]:
review_info = pd.DataFrame(items_comments,columns=['id_item','link','point','cnt','keyword'])

In [171]:
!ls data

item.csv        item_detail.csv review_info.csv


In [200]:
#review_info.to_csv('data/review_info.csv',index=False)

In [79]:
review_info = pd.read_csv('data/review_info.csv', thousands=',')

In [80]:
n_review_info = review_info[review_info['point'].notna()]
n_review_info = n_review_info.reset_index(drop=True)
n_review_info['cnt'] = n_review_info['cnt'].astype(int)

In [81]:
len(n_review_info)

2471

In [92]:
from tqdm import tqdm_notebook

In [220]:
def get_naver_review(url, cnt, iid, driver):
    #print(url)
    
    has_error = False
    item_info=[]
    rows = [] 
    try:
        driver.get(url) 
        lis = driver.find_elements_by_css_selector('ul.lst_review li:not(.u_likeit_list)')
        sleep(2)

        #print(len(lis), cnt)
        
        # 추가정보 가져오기
        item_infos = driver.find_elements_by_css_selector('div.info_inner.list span')
          
        lp = ''
        lowestp = driver.find_elements_by_css_selector('div.price_area .low_price .num')
        if len(lowestp)> 0: 
            lp = lowestp[0].text
        
        item_info.append(iid)
        item_info.append(lp)
        for iinfo in item_infos:
            if len(iinfo.text)>0:
                item_info.append(iinfo.text)
     
    except Exception as e:
        has_error = True
        print(iid, 'start error') 
        print('========================')
        
    
    if has_error:
        del item_info
        del rows
        item_info=[]
        rows = [] 
    else:
         
        oricnt = cnt
        isone = False
        if oricnt < 20: 
            isone=True
            
        #for page_seq in range(cnt//20 + 1): 
        
        page_ls = list(range(cnt//20 + 1))
        page_idx = 0
        while(True):
            page_seq = page_ls[page_idx] 
            try:
                for li in lis:  
                    html= li.get_attribute('innerHTML')
                    soup = BeautifulSoup(html) 

                    atc_area = soup.find(class_="atc_area")
                    avg_area = atc_area.find(class_='avg_area')

                    point = avg_area.find(class_='curr_avg').get_text()
                    site = avg_area.find(class_='path').get_text()
                    person = avg_area.find(class_='name').get_text()

                    date = avg_area.find(class_='date').get_text()
                    subj = atc_area.find(class_='subjcet').get_text() 
                    atc = atc_area.find(class_='atc').get_text()

                    img = ''
                    thmb = atc_area.find(class_='thmb')
                    if thmb: 
                        img = thmb.find('img', class_='_reviewImg')['data-original']
 
                    rows.append([iid, point, site, person, date, subj, atc, img]) 
                 
                if isone==False:
                    # 페이지 갱신
                    driver.execute_script("shop.detail.ReviewHandler.page({}, '_review_paging');".format(page_seq+2))
                    sleep(0.3)
                    lis = driver.find_elements_by_css_selector('ul.lst_review li:not(.u_likeit_list)')
             
                page_idx +=1
                if page_idx == len(page_ls):
                    break 
            except Exception as e: 
                print('li error')
                
                print(iid, len(rows), oricnt, page_idx, page_seq, len(lis)) 
                 
                page_idx +=1
                if page_idx == len(page_ls):
                    break 
                    
                page_seq = page_ls[page_idx] 
                
                driver.execute_script("shop.detail.ReviewHandler.page({}, '_review_paging');".format(page_seq+2))
                sleep(1)
                lis = driver.find_elements_by_css_selector('ul.lst_review li:not(.u_likeit_list)')
               
                #print(e) 
                print('========================')
      
        
    return rows, item_info


def get_naver_review_group(links, cnts, ids):
    
    driver =  webdriver.PhantomJS()
    for link, cnt, iid in tqdm_notebook(zip(links, cnts, ids), total=len(ids)):  
        naver_reviews, item_info= get_naver_review(link, cnt, iid, driver)
        all_naver_reviews.extend(naver_reviews)
        all_naver_iteminfos.append(item_info)
    driver.close() 

In [158]:
!killall phantomjs

In [144]:
driver =  webdriver.PhantomJS()

In [145]:
error = n_review_info[n_review_info.id_item==9708]

In [ ]:
res = get_naver_review(error.link.values[0], error.cnt.values[0], error.id_item.values[0], driver)
res

In [96]:
!killall phantomjs

No matching processes belonging to you were found


In [34]:
2471/ 3

823.6666666666666

In [189]:
all_naver_reviews = []
all_naver_iteminfos = []
for cnt in range(1, 4): 
    start = 823 * (cnt-1)
    end = 823 * cnt
    
    if cnt != 3:
        links = n_review_info.loc[start:end]['link'].values
        cnts = n_review_info.loc[start:end]['cnt'].values
        ids = n_review_info.loc[start:end]['id_item'].values 
    else :
        links = n_review_info.loc[start:]['link'].values
        cnts = n_review_info.loc[start:]['cnt'].values
        ids = n_review_info.loc[start:]['id_item'].values 
        print('end')
    
    print(start, end)
    th = threading.Thread(target=get_naver_review_group, args=(links, cnts, ids ))
    th.start()

0 823
823 1646
end
1646 2469


In [233]:
error_ids = [32997,28248,31333,29008,23710,13782,31165,22572,26857,22967,29941,31699,32904,32637,26826,28456,32681,27760,32680,30392,28859,28856,17359,25071,31914,32540,32974,31148,28428,28428,30959,17211,32537,32969,30957,29923,29994,29563,26990,27255,31878,22006,27926,32412,27822,23733,23732,26760,13297,27079,31725,32404,28001,26185,23314,25839,28162,27071,28271,27857,31779,15144,20364,25587,33179,28153 ]

In [78]:
error_ids2 =[29008,31165,25390,25390,22967,25390,22967,22967,29941,27882,28435,32904,26826,28521,32680,26857,17211,27255,27079,31699,28859,28151,33004,22572,28456,28456,32537,30308,23733,23732,32715,25587,14970,21877,21856,21826,31822,31822,27597,32010,32395,31876,32384,26667,28328 ]

In [150]:
error_ids3=[29008,31165,29941,26826,22572,28456,28456,23733,23732,25587,21826,27597,32010,25390,26857,31699,30308,32715,31822,31822,31822,32395,32384,26667 ]

In [237]:
complete_reviews = sample_reviews[~sample_reviews.id_item.isin(error_ids)]

In [242]:
notfinish = n_review_info[~n_review_info.id_item.isin(complete_reviews.id_item)]

In [244]:
notfinish.to_csv('data/notfinish.csv',index=False)

In [245]:
complete_reviews.to_csv('data/complete_reviews.csv',index=False)

In [15]:
notfinish = pd.read_csv('data/notfinish.csv')

In [85]:
notfinish2 = n_review_info[n_review_info.id_item.isin(error_ids2)]

In [151]:
notfinish3 = n_review_info[n_review_info.id_item.isin(error_ids3)]

In [96]:
!killall phantomjs

No matching processes belonging to you were found


In [36]:
all_naver_reviews = []
all_naver_iteminfos = []

start = 0
end = 372
links1 = notfinish.loc[start:end]['link'].values
cnts1 = notfinish.loc[start:end]['cnt'].values
ids1 = notfinish.loc[start:end]['id_item'].values 
th1 = threading.Thread(target=get_naver_review_group, args=(links1, cnts1, ids1 ))
th1.start()

start = 372 
links2 = notfinish.loc[start:]['link'].values
cnts2 = notfinish.loc[start:]['cnt'].values
ids2 = notfinish.loc[start:]['id_item'].values 
th2 = threading.Thread(target=get_naver_review_group, args=(links2, cnts2, ids2 ))
th2.start()


In [107]:
!killall phantomjs

No matching processes belonging to you were found


In [108]:
all_naver_reviews = []
all_naver_iteminfos = []
 
start = 0
end = 50
links1 = notfinish2.loc[start:end]['link'].values
cnts1 = notfinish2.loc[start:end]['cnt'].values
ids1 = notfinish2.loc[start:end]['id_item'].values 
th1 = threading.Thread(target=get_naver_review_group, args=(links1, cnts1, ids1 ))
th1.start()

start = 50 
links2 = notfinish2.loc[start:]['link'].values
cnts2 = notfinish2.loc[start:]['cnt'].values
ids2 = notfinish2.loc[start:]['id_item'].values 
th2 = threading.Thread(target=get_naver_review_group, args=(links2, cnts2, ids2 ))
th2.start()

In [144]:
len(all_naver_iteminfos), len(all_naver_reviews)

(107, 546604)

In [221]:
!killall phantomjs

No matching processes belonging to you were found


In [222]:
all_naver_reviews = []
all_naver_iteminfos = []
links1 = notfinish3.loc[:]['link'].values
cnts1 = notfinish3.loc[:]['cnt'].values
ids1 = notfinish3.loc[:]['id_item'].values 
get_naver_review_group(links1, cnts1, ids1)


li error
29008 1384 6011 69 69 5
li error
29008 4884 6011 245 245 20
li error
31165 5052 43926 252 252 20
li error
31165 5735 43926 287 287 20
li error
31165 6878 43926 345 345 20
li error
31165 9278 43926 466 466 20
li error
31165 10457 43926 525 525 20
li error
31165 15462 43926 776 776 20
li error
31165 17642 43926 886 886 20
li error
31165 18165 43926 913 913 20
li error
31165 28805 43926 1446 1446 20
li error
31165 31005 43926 1557 1557 20
li error
31165 31905 43926 1603 1603 20
li error
31165 34245 43926 1721 1721 20
li error
31165 37245 43926 1872 1872 20
li error
31165 42285 43926 2125 2125 20
li error
29941 2341 21811 117 117 20
li error
29941 4563 21811 229 229 20
li error
29941 8335 21811 418 418 20
li error
29941 8987 21811 451 451 20
li error
29941 13987 21811 702 702 20
li error
26826 813 18532 40 40 20
li error
26826 2478 18532 124 124 20
li error
26826 8695 18532 435 435 20
li error
26826 13795 18532 691 691 20
li error
22572 1275 42660 63 63 20
li error
22572 2340 426

In [ ]:
# 에러 다시 시작
# 안한거 다시 시작

In [135]:
len(all_naver_iteminfos), len(all_naver_reviews)

(106, 522384)

In [142]:
len(set(pd.DataFrame(all_naver_iteminfos)[0]))

39

In [139]:
len(set(notfinish2.id_item))

39

In [223]:
sample_reviews= pd.DataFrame(all_naver_reviews, columns=['id_item','point','site','author','date','sub','contents','img'])
sample_reviews

,id_item,point,site,author,date,sub,contents,img
0,28834,5,G마켓,pbr9****님,2014.12.23.,넘이뿌고 화려하고 빨리 건조되어 좋네요.,50대 후반인데 발라보니 넘 이뿌네요.색상이 화사하고 빨리 건조되어서 더욱더 좋네요...,
1,28834,5,GSSHOP,cjsy****님,2016.04.21.,색이은은해요..생각보다 반짝이는 잘 발리지않아요~~,색이은은해요..생각보다 반짝이는 잘 발리지않아요~~,
2,28834,4,SSG닷컴,swee****님,2016.03.26.,색상은 둘이 잘 어울리는데요. ?,색상은 둘이 잘 어울리는데요. 손톱에 손상을 많이 주는군요,
3,28834,4,신세계몰,swee****님,2016.03.26.,손톱에 손상을 많이 주는군요,손톱에 손상을 많이 주는군요,
4,28834,5,롯데닷컴,rmsc****님,2016.03.21.,색이 참 예뻐요.,색이 참 예뻐요.,
5,28834,5,GSSHOP,musi****님,2016.01.29.,가격 품질 모두 만족합니다.,가격 품질 모두 만족합니다.,
6,28834,5,GSSHOP,pink****님,2016.01.05.,서비스가 이래도 되나요.....♡아이브로우 카라와 매니큐어 하나 주문했을 뿐인데쇼핑백에,서비스가 이래도 되나요.....♡아이브로우 카라와 매니큐어 하나 주문했을 뿐인데쇼핑...,https://shopping-phinf.pstatic.net/20160106_2/...
7,28834,5,롯데아이몰,enab****님,2016.01.08.,맘에 들어요. 배송 빠르고 사은품도 감사.,맘에 들어요. 배송 빠르고 사은품도 감사.,
8,28834,5,롯데아이몰,fri2****님,2015.10.23.,아주 조아요. 배송도 빠르고요.,아주 조아요. 배송도 빠르고요.,
9,28834,5,롯데닷컴,leei****님,2015.10.10.,러블리로 사고 싶었지만 쉬크가 더 이쁘네여~색도...,러블리로 사고 싶었지만 쉬크가 더 이쁘네여~색도 예쁘고 좋으네여~,


In [76]:
import pickle

In [225]:
#with open('naver_reviews.plk','wb') as f:
#    pickle.dump(sample_reviews, f)

In [232]:
#item_d_info = pd.DataFrame(all_naver_iteminfos)
#item_d_info.to_csv('item_detail_naver.csv', index=False)

In [70]:
partcomplete = pd.DataFrame(all_naver_reviews, columns=['id_item','point','site','author','date','sub','contents','img'])
partcomplete

,id_item,point,site,author,date,sub,contents,img
0,30794,5,G마켓,new*****님,2016.12.14.,하루만에 온 배송! 저렴한 가격에 구입!,녹여쓰는 밤타입 스킨제품은 처음 쓰는데 화장하기전에 쓰면 화장이 안뜬다해서 겨울에 ...,https://shopping-phinf.pstatic.net/20170322_12...
1,30794,5,AK MALL,dmso****님,2016.12.21.,작지만 짐승용량,살짝 체온으로 녹여주면 매끈매끈 향도 좋은 오일로 변신하네요..덕분에 건조한 겨울을...,
2,30794,3,11번가,ki******님,2016.02.01.,나쁘지않아요.,워낙 유명해서 사봤는데 밤타입이라 녹여쓰는게 살짝 번거롭긴해도 잘쓰고 있어요. 아침...,
3,30794,3,내피부애,khjk****님,2018.04.28.,배송이5일이나걸리네요^^,배송이5일이나걸리네요^^,
4,30794,5,GSSHOP,kmar****님,2018.04.24.,피부에 잘 스며들고 오래감.,피부에 잘 스며들고 오래감.,
5,30794,4,위메프,bhy*****님,2018.04.13.,촉촉하고 부드럽고 광택나고 휴대하며 사용하기 좋아요,촉촉하고 부드럽고 광택나고 휴대하며 사용하기 좋아요,
6,30794,4,G마켓,godt****님,2018.03.30.,배송일과 물건상태,고요합니다.배송일이 2일 입니다.포장은 박스 포장으로 왔습니다.아직 사용을 안해봐서...,
7,30794,5,G마켓,secr****님,2018.03.21.,한율 진정밤,겨울이나 환절기에 필수품이에요,
8,30794,5,뷰티바이블,zizi****님,2018.01.28.,촉촉보단 오일리한듯요,수분감은 잘 모르겠어요 오일리한 느낌이라 크림바른후에 소량 사용중이에요 소량만으로도...,
9,30794,5,뷰티바이블,zizi****님,2018.01.27.,좋아용~~~^^,좋아용~~~^^,


In [71]:
error2 =[29008,31165,25390,25390,22967,25390,22967,22967,29941,27882,28435,32904,26826,28521,32680,26857,17211,27255,27079,31699,28859,28151,33004,22572,28456,28456,32537,30308,23733,23732,32715,25587,14970,21877,21856,21826,31822,31822,27597,32010,32395,31876,32384,26667,28328 ]

In [75]:
partcomplete = partcomplete[~partcomplete.id_item.isin(error2)]

In [77]:
with open('naver_reviews2.plk','wb') as f:
    pickle.dump(partcomplete, f)

In [89]:
item_d_info = pd.DataFrame(all_naver_iteminfos)
item_d_info.to_csv('item_detail_naver2.csv', index=False)

In [143]:
partcomplete = pd.DataFrame(all_naver_reviews, columns=['id_item','point','site','author','date','sub','contents','img'])
partcomplete

,id_item,point,site,author,date,sub,contents,img
0,29008,3,11번가,zi******님,2015.10.26.,저는 번져요 ㅠㅠ,두시간정도는 괜찮은데 두시간넘어가면슬슬 팬더로 변신합니다 힘들어요 ㅠㅠㅠㅠ저도 눈화...,
1,29008,5,SSG닷컴,hjh7****님,2015.10.05.,이거 대박이에요,바로 뜯어서 ?이거 대박이에요바로 뜯어서 손등에 그리고 문질렀는데 잘 안지워지고 잘...,
2,29008,4,SSG닷컴,sora****님,2014.09.19.,늘 쓰던 제품입니다~,펄브라운?늘 쓰던 제품입니다~펄브라운은 잘 번지지도 않고(쌍꺼풀이 없는 눈이라 아이...,
3,29008,5,11번가,so******님,2013.08.28.,"배송빼고,,,",8월 21일낧 주문해서 27일날 받았어요. 너무 늦은것만 빼고는 좋아요. 처음써본...,
4,29008,3,마이뷰티몰,skag****님,2018.04.29.,싸게사서좋긴한데 생각보다 제조일자가 오래됬네요. 박스없이 아이라이너만와서 제조일자 ...,싸게사서좋긴한데 생각보다 제조일자가 오래됬네요. 박스없이 아이라이너만와서 제조일자 ...,
5,29008,3,마이뷰티몰,skag****님,2018.04.29.,싸게 사서 좋긴한데요..,박스가 없이 와서 유통기한을 알수가 없었는데 유통기한 사이트에 검색해보니까 2016...,
6,29008,5,G마켓,viol****님,2018.04.29.,빠른배송 굿,잘받았어요 담날바로오네요ㅋㅋ,https://shopping-phinf.pstatic.net/20180501_11...
7,29008,5,11번가,fo******님,2018.04.13.,최고예요,써본 펜슬아이라이너중 가장 발색이 선명하고 번짐없어서 이것만 계속 쓰네요 ~ 최고,
8,29008,5,11번가,de******님,2013.11.14.,4개째 쓰고 있는 제품..,몇년째 쓰고 있는 제품입니다.잃어 버린 줄 알고 다시 샀는데..;; 찾았네요.;펜슬...,
9,29008,2,인터파크,pok*****님,2018.01.30.,별로에요,그 전버전 쓰다가 이걸로 바꿔봤는데 번짐있고 별로네요 ㅠㅠㅠㅠ,


In [145]:
error3=[29008,31165,29941,26826,22572,28456,28456,23733,23732,25587,21826,27597,32010,25390,26857,31699,30308,32715,31822,31822,31822,32395,32384,26667 ]

In [146]:
partcomplete = partcomplete[~partcomplete.id_item.isin(error3)]

In [147]:
with open('naver_reviews3.plk','wb') as f:
    pickle.dump(partcomplete, f)

In [148]:
item_d_info = pd.DataFrame(all_naver_iteminfos)
item_d_info.to_csv('item_detail_naver3.csv', index=False)

In [223]:
partcomplete = pd.DataFrame(all_naver_reviews, columns=['id_item','point','site','author','date','sub','contents','img'])
partcomplete

,id_item,point,site,author,date,sub,contents,img
0,29008,3,11번가,zi******님,2015.10.26.,저는 번져요 ㅠㅠ,두시간정도는 괜찮은데 두시간넘어가면슬슬 팬더로 변신합니다 힘들어요 ㅠㅠㅠㅠ저도 눈화...,
1,29008,5,SSG닷컴,hjh7****님,2015.10.05.,이거 대박이에요,바로 뜯어서 ?이거 대박이에요바로 뜯어서 손등에 그리고 문질렀는데 잘 안지워지고 잘...,
2,29008,4,SSG닷컴,sora****님,2014.09.19.,늘 쓰던 제품입니다~,펄브라운?늘 쓰던 제품입니다~펄브라운은 잘 번지지도 않고(쌍꺼풀이 없는 눈이라 아이...,
3,29008,5,11번가,so******님,2013.08.28.,"배송빼고,,,",8월 21일낧 주문해서 27일날 받았어요. 너무 늦은것만 빼고는 좋아요. 처음써본...,
4,29008,3,마이뷰티몰,skag****님,2018.04.29.,싸게사서좋긴한데 생각보다 제조일자가 오래됬네요. 박스없이 아이라이너만와서 제조일자 ...,싸게사서좋긴한데 생각보다 제조일자가 오래됬네요. 박스없이 아이라이너만와서 제조일자 ...,
5,29008,3,마이뷰티몰,skag****님,2018.04.29.,싸게 사서 좋긴한데요..,박스가 없이 와서 유통기한을 알수가 없었는데 유통기한 사이트에 검색해보니까 2016...,
6,29008,5,G마켓,viol****님,2018.04.29.,빠른배송 굿,잘받았어요 담날바로오네요ㅋㅋ,https://shopping-phinf.pstatic.net/20180501_11...
7,29008,5,11번가,fo******님,2018.04.13.,최고예요,써본 펜슬아이라이너중 가장 발색이 선명하고 번짐없어서 이것만 계속 쓰네요 ~ 최고,
8,29008,5,11번가,de******님,2013.11.14.,4개째 쓰고 있는 제품..,몇년째 쓰고 있는 제품입니다.잃어 버린 줄 알고 다시 샀는데..;; 찾았네요.;펜슬...,
9,29008,2,인터파크,pok*****님,2018.01.30.,별로에요,그 전버전 쓰다가 이걸로 바꿔봤는데 번짐있고 별로네요 ㅠㅠㅠㅠ,


In [224]:
with open('naver_reviews4.plk','wb') as f:
    pickle.dump(partcomplete, f)

In [225]:
item_d_info = pd.DataFrame(all_naver_iteminfos)
item_d_info.to_csv('item_detail_naver4.csv', index=False)